# Running Pre-Trained CGNet Models

Purpose:
--------
The purpose of this notebook is to run pre-trained CGnet models for machine learning detection of atmospheric rivers and tropical cyclones.\
See ClimateNet repo here: https://github.com/andregraubner/ClimateNet

Authors/Contributors:
---------------------
* Teagan King
* John Truesdale
* Katie Dagon

## Import libraries

In [1]:
import os
import sys
import numpy as np

sys.path.append("/glade/work/kdagon/ClimateNet") # append path to ClimateNet repo
from climatenet.utils.data import ClimateDatasetLabeled, ClimateDataset
from climatenet.models import CGNet
from climatenet.utils.utils import Config
from climatenet.track_events import track_events
from climatenet.analyze_events import analyze_events
from climatenet.visualize_events import visualize_events

from os import path

## Functions for setting up configurations, using pre-trained model, and creating TC/AR masks

In [2]:
def load_model_and_create_masks(year, inference_path, save_dir, config, cgnet, analyze=False, visualize=False):
    """Load a pre-trained model and use it to create masks of tropical cyclones (mask value = 1)
    and atmospheric rivers (mask value = 2)
    
    Function will create NetCDF mask files and save them in the save_directory.

    Parameters:
    -----------
    year: int
        year at which to run model and create masks
    inference_path: str
        filepath to inference data. It is expected that the year is at the end of the filepath in the format ####
    save_dir: str
        filepath to where the masks will be saved as .nc files
    config: Config object
        output from setup_climatenet_config()
    cgnet: CGNet object
        output from setup_climatenet_config()
    analyze: bool (optional)
        default is False; if True, will save plots for analyzing events using climatenet.analyze_events().
        Note that this can significantly increase the time to run.
    visualize : bool (optional)
        default is False; if True, will save plots for visualizing events using climatenet.visualize_events().
        Note that this can significantly increase the time to run.
    
    """
    if not os.path.isdir(save_dir+str(year)):
        os.mkdir(save_dir+str(year))
    else:
        print("Warning: might overwrite {}".format(save_dir+str(year)))
    inference = ClimateDataset(inference_path+str(year), config)
    class_masks = cgnet.predict(inference) # masks with 1==TC, 2==AR
    class_masks.to_netcdf(save_dir+str(year)+"/class_masks."+str(year)+".nc")
    event_masks = track_events(class_masks) # masks with event IDs
    event_masks.to_netcdf(save_dir+str(year)+"/event_masks."+str(year)+".nc")
    if analyze:
        analyze_events(event_masks, class_masks, save_dir+str(year)+"/")
    if visualize:
        visualize_events(event_masks, inference, save_dir+str(year)+"/")
    print("Saved class and event masks to {}".format(save_dir+str(year)))

    return

In [3]:
def setup_climatenet_config(config_path, trained_model):
    """Set up configuration file and load in a pre-trained model
    
    Parameters:
    -----------
    config_path: str
        filepath to configuration file
    trained_model: str
        filepath to a pre-trained model
    """

    config = Config(config_path)
    cgnet = CGNet(config)
    cgnet.load_model(trained_model)
    cgnet.network.eval() # gives evaluation stats

    return config, cgnet

## Load pre-trained model which uses TMQ, V850, U850, and PSL

In [4]:
config = Config('/glade/work/tking/cgnet/trained_cgnet.042922/config.json')  # tmq, v850, u850, and psl
cgnet = CGNet(config)
cgnet.load_model('/glade/work/tking/cgnet/trained_cgnet.042922')  # tmq, v850, u850, and psl
cgnet.network.eval() # gives evaluation stats

# for year in range(2001, 2015):
#     if not os.path.isdir("/glade/scratch/tking/results.tmq_psl_v850_u850.{}".format(year)):
#         os.mkdir("/glade/scratch/tking/results.tmq_psl_v850_u850.{}".format(year))
#     inference_path = "/glade/campaign/cgd/amp/jet/ClimateNet/Climate_data_"+str(year)
#     inference = ClimateDataset(inference_path, config)
#     class_masks = cgnet.predict(inference) # masks with 1==TC, 2==AR
#     class_masks.to_netcdf("/glade/scratch/tking/results.tmq_psl_v850_u850."+str(year)+"/class_masks."+str(year)+".test.nc")
#     event_masks = track_events(class_masks) # masks with event IDs
#     event_masks.to_netcdf("/glade/scratch/tking/results.tmq_psl_v850_u850."+str(year)+"/event_masks."+str(year)+".test.nc")
#     analyze_events(event_masks, class_masks, "/glade/scratch/tking/results.tmq_psl_v850_u850."+str(year)+"/")
#     # visualize_events(event_masks, inference, "/glade/scratch/tking/results.tmq_psl_v850_u850."+str(year)+"/")               

RuntimeError: cuda runtime error (38) : no CUDA-capable device is detected at /opt/conda/conda-bld/pytorch_1565272279342/work/aten/src/THC/THCGeneral.cpp:50

In [ ]:
# %%time

# # parameters for setup_climatenet_config()
# config_path = '/glade/u/home/tking/cgnet/ClimateNet/config.json'
# trained_model = '/glade/u/home/tking/cgnet/trained_cgnet.042922'

# # parameters for load_model_and_create_masks()
# inference_path = "/glade/campaign/cgd/amp/jet/ClimateNet/data_processing/extend/2006-2015-RCP26-3hr/"  # expects year as #### format at end of data directory
# save_dir = "/glade/scratch/tking/results.tmq_psl_v850_u850."  # expects year as #### format at end of data directory


# config, cgnet = setup_climatenet_config(config_path, trained_model)
# for year in range(2006, 2015):
#     load_model_and_create_masks(year, inference_path, save_dir, config, cgnet)

In [ ]:
# %%time

# # parameters for setup_climatenet_config()
# config_path = '/glade/u/home/tking/cgnet/ClimateNet/config.json'
# trained_model = '/glade/u/home/tking/cgnet/trained_cgnet.042922'

# # parameters for load_model_and_create_masks()
# inference_path = "/glade/campaign/cgd/amp/jet/ClimateNet/data_processing/extend/2006-2015-RCP26-3hr/"  # expects year as #### format at end of data directory
# save_dir = "/glade/scratch/tking/results.tmq_psl_v850_u850."  # expects year as #### format at end of data directory


# config, cgnet = setup_climatenet_config(config_path, trained_model)
# for year in range(2013, 2015):
#     load_model_and_create_masks(year, inference_path, save_dir, config, cgnet)

In [ ]:
# %%time

# # parameters for setup_climatenet_config()
# config_path = '/glade/u/home/tking/cgnet/ClimateNet/config.json'
# trained_model = '/glade/u/home/tking/cgnet/trained_cgnet.042922'

# # parameters for load_model_and_create_masks()
# inference_path = "/glade/campaign/cgd/amp/jet/ClimateNet/data_processing/extend/2006-2015-RCP26-3hr/"  # expects year as #### format at end of data directory
# save_dir = "/glade/scratch/tking/results.tmq_psl_v850_u850."  # expects year as #### format at end of data directory


# config, cgnet = setup_climatenet_config(config_path, trained_model)
# load_model_and_create_masks(2015, inference_path, save_dir, config, cgnet)

# Load pre-trained model which uses TMQ and V850

In [9]:
# parameters for setup_climatenet_config()
config_path = '/glade/work/tking/cgnet/ClimateNet/config.json'
trained_model = '/glade/work/tking/cgnet/trained_cgnet.tmq_v850.092822'

# parameters for load_model_and_create_masks()
inference_path = "/glade/campaign/cgd/amp/jet/ClimateNet/Climate_data_"  # expects year as #### format at end of data directory
save_dir = "/glade/scratch/tking/results.tmq_v850."  # expects year as #### format at end of data directory

In [9]:
%%time
config, cgnet = setup_climatenet_config(config_path, trained_model)

for year in range(2001, 2006):
    load_model_and_create_masks(year, inference_path, save_dir, config, cgnet)

100%|██████████| 183/183 [06:02<00:00,  1.98s/it]


identifying connected components..
tracking components across time..


100%|██████████| 2920/2920 [45:56<00:00,  1.06it/s]


num TCs: 364
num ARs: 554


  0%|          | 0/183 [00:00<?, ?it/s]

...

Saved class and event masks to /glade/scratch/tking/results.tmq_v850.2004


100%|██████████| 183/183 [06:02<00:00,  1.98s/it]


identifying connected components..
tracking components across time..


100%|██████████| 2920/2920 [47:22<00:00,  1.03it/s]


num TCs: 322
num ARs: 534
Saved class and event masks to /glade/scratch/tking/results.tmq_v850.2005
CPU times: user 4h 6min 20s, sys: 12min 33s, total: 4h 18min 53s
Wall time: 5h 40min 49s


In [10]:
%%time
config, cgnet = setup_climatenet_config(config_path, trained_model)
inference_path = "/glade/campaign/cgd/amp/jet/ClimateNet/data_processing/extend/2006-2015-RCP26-3hr/"  # expects year as #### format at end of data directory

# for year in range(2006, 2016):
for year in range(2011, 2016):
    load_model_and_create_masks(year, inference_path, save_dir, config, cgnet)

  0%|          | 0/183 [00:00<?, ?it/s]

100%|██████████| 183/183 [09:25<00:00,  3.09s/it]


identifying connected components..
tracking components across time..


100%|██████████| 2920/2920 [45:44<00:00,  1.06it/s]


num TCs: 370
num ARs: 522


...

Saved class and event masks to /glade/scratch/tking/results.tmq_v850.2014


100%|██████████| 183/183 [08:34<00:00,  2.81s/it]


identifying connected components..
tracking components across time..


100%|██████████| 2920/2920 [47:09<00:00,  1.03it/s]


num TCs: 414
num ARs: 499
Saved class and event masks to /glade/scratch/tking/results.tmq_v850.2015
CPU times: user 4h 4min 27s, sys: 12min 32s, total: 4h 16min 59s
Wall time: 5h 51min 15s


In [ ]:
# for year in range(2086, 2100):
#     load_model_and_create_masks(year, inference_path, save_dir, config, cgnet)


# Load pre-trained model which uses TMQ only

In [5]:
# parameters for setup_climatenet_config()
config_path_tmq = "/glade/work/tking/cgnet/ClimateNet/config.json"
trained_model_tmq = "/glade/work/tking/cgnet/trained_cgnet.tmq_only.091922"  # USING NEW MODEL WITH CORRECT CONFIGS

# parameters for load_model_and_create_masks()
# up to 2005:
# inference_path = "/glade/campaign/cgd/amp/jet/ClimateNet/Climate_data_"  # expects year as #### format at end of data directory
# after 2006:
inference_path = "/glade/campaign/cgd/amp/jet/ClimateNet/data_processing/extend/2006-2015-RCP26-3hr/"
save_dir_tmq = "/glade/scratch/tking/results.tmq_only."  # expects year as #### format at end of data directory

In [6]:
%%time
config, cgnet = setup_climatenet_config(config_path_tmq, trained_model_tmq)

CPU times: user 1.52 s, sys: 881 ms, total: 2.4 s
Wall time: 5.36 s


In [7]:
# for year in range(2000, 2005):
# for year in range(2006, 2011):
# for year in range(2013, 2015):
#     load_model_and_create_masks(year, inference_path, save_dir_tmq, config, cgnet)
# load_model_and_create_masks(2015, inference_path, save_dir_tmq, config, cgnet)

100%|██████████| 183/183 [08:45<00:00,  2.87s/it]


identifying connected components..
tracking components across time..


100%|██████████| 2920/2920 [41:26<00:00,  1.17it/s]


num TCs: 263
num ARs: 428
Saved class and event masks to /glade/scratch/tking/results.tmq_only.2015


In [1]:
ls /glade/p/cgd/ccr/people/tking/cgnet_masks/

tmq_only/  tmq_psl_v850_u850/  tmq_v850/


In [12]:
# ls /glade/scratch/tking/results.tmq_only*/*mask*